# BioArtBot Artistic Procedure
This file contains a procedure that the Opentrons OT2 can use to make the art you just collected into a _reality!_ This notebook style is a good choice if you have a lot of artpieces, or need to troubleshoot, because you get to run the code incrementally.

If you haven't done so already, connect to the Jupyter Notbook server of your OT2 at ```[Robot's IP address]:48888``` in your browser. Full docs on this can be found [here](https://support.opentrons.com/en/articles/1795367-running-the-robot-using-jupyter-notebook) From the main page of the Jupyter server, click "Upload" in the top-right, and upload this file.

After that, you just need to run each cell in order (click inside cell, then ```shift+enter```). The cells are broken up so you can confirm each worked before progressing to the next one. **The robot won't move until you run the last cell**. For best results, make sure you have calibrated the robot for each type of lab equipment you'll use first. The OT2 App can walk you through this.

### Cell 1: Setup and connect to robot
Estimated time: almost immediate

In [ ]:
from opentrons import labware, instruments, robot
import math

robot.connect()

### Cell 2: Define labware and pipetting method
Estimated time: a few seconds at most

In [ ]:
metadata = {
    'protocolName': 'CCL ARTBot',
    'author': 'Tim Dobbs and Counter Culture Labs',
    'source': 'ARTBot Protocol Builder'
    }

def distribute_to_agar(self, vol, source, destination, disposal_vol):

    self.max_volume = 20
    
    dest = list(destination) #allows for non-lists

    for cnt, well in enumerate(dest):
        if (cnt + 1) % 150 == 0:
            self.drop_tip()
            self.current_volume = 0
        
        if not self.current_tip():
            self.pick_up_tip()

        if self.current_volume < (vol + disposal_vol):
            remaining_wells = len(dest) - cnt
            remaining_vol = remaining_wells * vol

            if remaining_vol + disposal_vol > self.max_volume:
                asp_vol = math.floor((self.max_volume - disposal_vol) / vol) * vol + disposal_vol - self.current_volume
            else:
                asp_vol =  remaining_vol + disposal_vol - self.current_volume

            self.aspirate(asp_vol, source)

        self.dispense(vol, well, 0.25)

        dip_position = robot._driver.position
        dip_position['Z'] = dip_position['Z'] - 11
        robot._driver.move(dip_position)

    self.drop_tip()

wellplate_type = %%WELLPLATE TYPE GO HERE%%
# a well plate for all of the colors in our pallette
palette = labware.load(wellplate_type, 11)

# a tip rack for our pipette
p200rack = labware.load('tiprack-200ul', 10)

# colored culture locations
palette_colors = dict(
    pink = palette.wells('A1')
    ,blue = palette.wells('A2')
    ,teal = palette.wells('A3')
    ,orange = palette.wells('B1')
    ,yellow = palette.wells('B2')
    )

p300 = instruments.P300_Single(
            mount='left',
            tip_racks=[p200rack]
    )

### Cell 3: Define pixel and canvas locations
_This part is unique to each set of artpieces you make_  
Estimated time: almost immediate

In [ ]:
pixels_by_color_by_artpiece = %%PIXELS GO HERE%%
canvas_locations = %%CANVAS LOCATIONS GO HERE%%

### Cell 4: Define "canvas" labware objects
Estimated time: This can take a very long time. ~5 minutes per artpiece

In [ ]:
# plates to create art in
canvas_labware = dict()
for art_title in canvas_locations:
    print('Making canvas for ' + art_title)
    canvas_labware[art_title] = labware.load('CCL_ARTBot_canvas', canvas_locations[art_title])

### Cell 5: \* Run the procedure \*
Activate this last one only when you're ready for the robot to start  
Estimated time: Depends on how many pixels are in the art, but figure on at least 20min per artpiece

In [ ]:
# wells to dispense each color material to
pixels_by_color = dict()
for color in pixels_by_color_by_artpiece:
    pixels_by_color[color] = list()
    pixels_by_artpiece = pixels_by_color_by_artpiece[color]
    for art_title in pixels_by_artpiece:
        pixels_by_color[color] += [
            pixel.top() for pixel in canvas_labware[art_title].wells(pixels_by_artpiece[art_title])
        ]
        if not len(canvas_labware[art_title].wells(pixels_by_artpiece[art_title])): #single-well case
            pixels_by_color[color] += [canvas_labware[art_title].wells(pixels_by_artpiece[art_title]).top()]

print('Reorganization of pixel dict complete')
            
def run_custom_protocol():

    print('starting colors')
    
    for color in pixels_by_color:
        print('Distributing ' + color)
        distribute_to_agar(p300, 0.1, palette_colors[color], pixels_by_color[color], disposal_vol=4)

print('Beginning protocol')
        
run_custom_protocol()